In [1]:
input_file = "input_files/day_19.txt"

with open(input_file) as lines:
    s = lines.read().strip()
    
raw_instructions, raw_parts = s.split('\n\n')
raw_instructions = raw_instructions.split('\n')
raw_parts = raw_parts.split('\n')


## Parse Input

In [2]:
def parse_part(s):
    kv = s[1:-1].split(',')
    d = {}
    for i in kv:
        k, v = i.split('=')
        d[k] = int(v)
    return d

def parse_instructions(s, instruction_constructor):
    key, inst = s.split('{')
    rule = inst[:-1].split(',')
    rule = [instruction_constructor(r) for r in rule]
    return key, rule
                    
def make_instruction_part_one(s):
    ''' 
    turns an instruction like `a<2006:qkq` into
    a lambda function that either returns the destination 
    or None if the conditions does not pass
    '''
    if ':' not in s:
        return lambda _: s
    
    i, dest = s.split(':')
    key = i[0]
    op = i[1]
    n = int(i[2:])

    if op == '<':
        return lambda d: dest if d[key] < n else None
    if op == '>':
        return lambda d: dest if d[key] > n else None

    
parts = [parse_part(p) for p in raw_parts]
workflows = dict([parse_instructions(i, make_instruction_part_one) for i in raw_instructions])


## Part One

In [3]:
def validate(p, workflows):
    work_iter = iter(workflows['in'])
    while True:
        f = next(work_iter)
        r = f(p)
        if r == 'A':
            return True
        if r == 'R':
            return False
        if r is None:
            continue
        else:
            work_iter = iter(workflows[r])

sum(sum(p.values()) for p in  parts if validate(p, workflows))
 

432427

## Part Two

In [4]:
def make_instruction_part_two(s):
    ''' 
    dict keyed to same key value is ranges accepted
    {a: [range(0, 45), range(3000, 4000)]
    '''
    if ':' not in s:
        return lambda d: ((s, d), )
    
    i, dest = s.split(':')
    k = i[0]
    op = i[1]
    n = int(i[2:])

    if op == '<':
        return lambda d: (
            (dest, {**d, k: d[k].intersect(Range(1, n))} ), 
            (None, {**d, k: d[k].intersect(Range(n, 4001))} )
        )
    if op == '>':
        return lambda d: (
            (dest, {**d, k: d[k].intersect(Range(n+1, 4001))} ),
            (None, {**d, k: d[k].intersect(Range(1, n+1))} )
        )
    
workflows = dict([parse_instructions(i, make_instruction_part_two) for i in raw_instructions])


In [5]:
from typing import NamedTuple
from math import prod

class Range(NamedTuple):
    start: int
    stop: int
    
    def intersect(self, other):
        if other is None:
            return None
        if max(self.start, other.start) <= min(self.stop, other.stop):
            return Range(max(self.start, other.start), min(self.stop, other.stop))
        
def validate(summary, workflows):
    accepted = []
    rejected = []
    work_iters = [('in', summary)]
    
    while len(work_iters):
        workflow_key, d = work_iters.pop()
        
        work_iter = iter(workflows[workflow_key])
        
        for f in  work_iter:
            f_result = f(d)
         
            for (r, res_dict) in f_result:
                if r == 'A':
                    accepted.append(res_dict)    
                elif r == 'R':
                    rejected.append(res_dict)
                elif r is None:
                    d = res_dict
                else:
                    work_iters.append((r, res_dict))
                                        
    return accepted, rejected              

start_solid = {
    'x': Range(1, 4001),
    'm': Range(1, 4001),
    'a': Range(1, 4001),
    's': Range(1, 4001),
}

accepted, rejected = validate(start_solid, workflows)

sum([prod(v.stop - v.start for v in d.values()) for d in accepted])


143760172569135